In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import gdown
import shutil
from pathlib import Path
from IPython.display import clear_output
Gdrive = False # IF you want to save data of images keep true
folder_path = '/content/drive/MyDrive/' if Gdrive else '/content/'
weights_Dir = Path('/content/drive/MyDrive/Colab_zip/multimodel_effnet/weights')
weights_Dir.mkdir(parents=True, exist_ok=True)

In [ ]:
#Config:
file_id =  None #'<add_you_coco_dataset_dive_link>'  #Eg. '1fdgdgssytsyusrtyshgg' ensure data to be zip
file_id_weigths = None #'<trained_weights any>' #Eg. '1fdgdgsgsjusysetg' : format: model_weights_00_100.pth
file_csv = None #<add csv drive id >
file_train_csv = None
file_test_csv = None

In [4]:
%%bash
cd /content
cd "$folder_path"  #goto location
git clone https://github.com/omer770/multimodel_effnet.git
cd multimodel_effnet
pip install -r requirements.txt -q
cd weights
wget -q https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth #default weights
cd /content/

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.4/314.4 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 86.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

Cloning into 'multimodel_effnet'...
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 3.7.4 requires typer<0.10.0,>=0.3.0, but you have typer 0.12.3 which is incompatible.
weasel 0.3.4 requires typer<0.10.0,>=0.3.0, but you have typer 0.12.3 which is incompatible.


In [5]:
if file_id:
  zip_path = gdown.download(id=file_id)
  !unzip -q {zip_path} -d {folder_path+'multimodel_effnet/data'}
  #!rm zip_path
  file_id = None
if file_csv:
  csv_name = gdown.download(id=file_csv)
  shutil.move(csv_name, folder_path+'multimodel_effnet/data')
  file_csv = None
if file_train_csv:
  train_csv_name = gdown.download(id=file_train_csv)
  shutil.move(train_csv_name, folder_path+'multimodel_effnet/data')
  file_train_csv = None
if file_test_csv:
  test_csv_name = gdown.download(id=file_test_csv)
  shutil.move(test_csv_name, folder_path+'multimodel_effnet/data')
  file_test_csv = None
if file_id_weigths:
  weights_path = gdown.download(id=file_id_weigths)
  try:shutil.move(weights_path, weights_Dir)
  except:print(f"{weights_path} path exits")
clear_output()
print('Completed')

Completed


In [6]:
%pwd

'/content'

In [7]:
%cd {folder_path+'multimodel_effnet'}

/content/multimodel_effnet


In [8]:
import os
import torch
import torchvision
import pandas as pd
import gradio as gr
from tqdm.auto import tqdm
from pathlib import Path
import torch.optim as optim
from torchinfo import summary
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.utils import save_image
from torchvision.models import efficientnet_b0,EfficientNet_B0_Weights

In [9]:
from test_runs import make_predictions,model_test
from multimodel_effnet.Efficientnet.util.train_utils import train
from multimodel_effnet.Efficientnet.model import multimodel_effnet_arc
from multimodel_effnet.Efficientnet.util.dataset import CustomImageDataset,create_balanced_df,WeightedSampler
from multimodel_effnet.Efficientnet.util.metrics_utils import cross_entropy_loss_embedded,accuracy_embedded
from multimodel_effnet.Efficientnet.util.visualize import plot_loss_curves,plot_test_results
from multimodel_effnet.Efficientnet.util.metrics_utils import calculate_ConfusionMatrices
from multimodel_effnet.Efficientnet.util.visualize import plot_confusion_matrices,display_random_images

In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"
#torch.set_default_device(device)
print("device: ",device)
base_path = Path(folder_path)
csv_path = base_path/'multimodel_effnet'/'data'/ csv_name
train_csv_path = base_path/'multimodel_effnet'/'data'/ train_csv_name
test_csv_path = base_path/'multimodel_effnet'/'data'/ test_csv_name
image_directory = '/content/multimodel_effnet/data/Datasets2'
weights_Dir = Path('/content/drive/MyDrive/Colab_zip/multimodel_effnet/weights')
weights_Dir.mkdir(parents=True, exist_ok=True)
df = pd.read_csv(csv_path)
attribs_m = ['Roof Condition', 'Roof Material','Roof Style','Solar Panel', 'Tree Overhang', 'Swimming Pool']
item_2_label_lst = [
    {'Fair':0, 'Good':1, 'Poor':2,'Damaged':3},
    {'Metal':0, 'Poly':1, 'Shingle':2, 'Tile':3,'Ballasted':4,'Asphalt':5},
    {'Flat':0, 'Gabled':1, 'Hip':2, 'Mixed':3},
    {'No':0,'Yes':1},
    {'Low':0, 'Medium':1,'High':2, 'No':3},
    {'No':0, 'Yes':1,'Screened':2}]
label_2_item_lst = [
    {0: 'Fair', 1: 'Good', 2: 'Poor', 3:'Damaged'},
    {0: 'Metal', 1: 'Poly', 2: 'Shingle', 3: 'Tile', 4: 'Ballasted', 5:'Asphalt'},
    {0: 'Flat', 1: 'Gabled', 2: 'Hip', 3: 'Mixed'},
    {0: 'No', 1:'Yes'},
    {0: 'Low', 1: 'Medium', 2:'High', 3: 'No'},
    {0: 'No', 1: 'Yes', 2:'Screened'}]
#print(df.head())

device:  cpu


In [11]:
weights = EfficientNet_B0_Weights.DEFAULT # .DEFAULT = best available weights
effnet = efficientnet_b0
model = multimodel_effnet_arc.effnet_multimodel(model_name='b1',effnet=effnet, weights =weights)
# Freeze all base layers in the "features" section of the model (the feature extractor) by setting requires_grad=False
for param in model.effnet.parameters():
    param.requires_grad = False

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 121MB/s]


In [12]:
filePaths = [file for file in weights_Dir.iterdir() if file.name.startswith('me_model_weights')]
try:
  latest_weigths = str(filePaths[-1])
  model.load_state_dict(torch.load(latest_weigths,map_location= device))
  print("choosen weights: ",latest_weigths)
except:
  latest_weigths= None
  print("choosen weights: ",latest_weigths)

choosen weights:  /content/drive/MyDrive/Colab_zip/multimodel_effnet/weights/me_model_weights_07_005.pth


In [13]:
# Augment train data
train_transforms = transforms.Compose([
    transforms.CenterCrop(768),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         #std=[0.229, 0.224, 0.225])
])

# Don't augment test data, only reshape
test_transforms = transforms.Compose([
    transforms.CenterCrop(768),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         #std=[0.229, 0.224, 0.225]),
])

In [14]:
df_test = pd.read_csv(test_csv_path)
df_test.head()

,Ortho Image,Image_ID,Roof Condition,Roof Material,Solar Panel,Roof Style,Tree Overhang,Swimming Pool
0,https://image.omniearth.net/api/imagery/v4/ima...,Image_1802_Ballasted_Flat_Fair.jpg,Fair,Ballasted,No,Flat,Low,Yes
1,https://image.omniearth.net/api/imagery/v4/ima...,Image_16500_Ballasted_Flat_Fair.jpg,Fair,Ballasted,No,Flat,No,No
2,https://image.omniearth.net/api/imagery/v4/ima...,Image_18425_Ballasted_Flat_Fair.jpg,Fair,Ballasted,No,Flat,No,No
3,https://image.omniearth.net/api/imagery/v4/ima...,Image_2425_Shingle_Mixed_Fair.jpg,Fair,Shingle,No,Mixed,Low,Screened
4,https://image.omniearth.net/api/imagery/v4/ima...,Image_10766_Shingle_Mixed_Fair.jpg,Fair,Shingle,No,Mixed,No,Screened


In [17]:
# Make predictions across test dataset with EffNetB2
effnetb2_test_pred_dicts = pred_and_store(model=model,
                                          transform=test_transforms,
                                          image_dir = image_directory,
                                          annotations_file=test_csv_path,
                                          device="cpu") # make predictions on CPU

  0%|          | 0/1006 [00:00<?, ?it/s]

In [18]:
# Inspect the first 2 prediction dictionaries
effnetb2_test_pred_dicts[:2]

[{'image_path': '/content/multimodel_effnet/data/Datasets2/Image_1802_Ballasted_Flat_Fair.jpg',
  'classes': ['Fair', 'Ballasted', 'Flat', 'No', 'Low', 'Yes'],
  'pred_classes': ['Fair', 'Ballasted', 'Flat', 'No', 'Low', 'Yes'],
  'time_for_pred': 0.6292,
  'correct': True},
 {'image_path': '/content/multimodel_effnet/data/Datasets2/Image_16500_Ballasted_Flat_Fair.jpg',
  'classes': ['Fair', 'Ballasted', 'Flat', 'No', 'No', 'No'],
  'pred_classes': ['Fair', 'Ballasted', 'Flat', 'No', 'No', 'No'],
  'time_for_pred': 0.0893,
  'correct': True}]

In [ ]:
# Turn the test_pred_dicts into a DataFrame
import pandas as pd
effnetb2_test_pred_df = pd.DataFrame(effnetb2_test_pred_dicts)
effnetb2_test_pred_df

In [46]:
example_list = [['/content/multimodel_effnet/data/Datasets2/Image_10766_Shingle_Mixed_Fair.jpg'],
                ['/content/multimodel_effnet/data/Datasets2/Image_1277_Tile_Mixed_Damaged.jpg'],
                ['/content/multimodel_effnet/data/Datasets2/Image_6229_Metal_Gabled_Poor.jpg'],
                ['/content/multimodel_effnet/data/Datasets2/Image_27753_Tile_Mixed_Fair.jpg'],
                ['/content/multimodel_effnet/data/Datasets2/Image_25246_Metal_Gabled_Good.jpg']]

In [20]:
# Check number of correct predictions
effnetb2_test_pred_df.correct.value_counts()

correct
False    837
True     169
Name: count, dtype: int64

In [ ]:
#effnetb2_test_pred_df.score.value_counts()

score
5    329
4    301
6    169
3    153
2     51
1      3
Name: count, dtype: int64

In [21]:
# Find the average time per prediction
effnetb2_average_time_per_pred = round(effnetb2_test_pred_df.time_for_pred.mean(), 4)
print(f"EffNetB2 average time per prediction: {effnetb2_average_time_per_pred} seconds")

EffNetB2 average time per prediction: 0.0919 seconds


In [22]:
# Put EffNetB2 on CPU
model.to("cpu")

# Check the device
next(iter(model.parameters())).device

device(type='cpu')

In [40]:
import random
from PIL import Image

# Get a list of all test image filepaths
test_data_paths = effnetb2_test_pred_df['image_path'].sort_values().values.tolist()
# Randomly select a test image path
random_image_path = random.sample(test_data_paths, k=1)[0]

# Open the target image
image = Image.open(random_image_path).convert("RGB")
print(f"[INFO] Predicting on image at path: {random_image_path}\n")


# Get results from the predict function
#predictions, confidences = predict(img=image)
print(predict(img=image,model=model,transform = test_transforms))

[INFO] Predicting on image at path: /content/multimodel_effnet/data/Datasets2/Image_12428_Metal_Gabled_Damaged.jpg

({'Fair': 0.62}, {'Poly': 0.37}, {'Flat': 0.64}, {'No': 1.0}, {'No': 0.66}, {'No': 0.97})


In [47]:
# Create a list of example inputs to our Gradio demo
eg_list = [[str(filepath)] for filepath in random.sample(test_data_paths, k=3)]
example_list

[['/content/multimodel_effnet/data/Datasets2/Image_10766_Shingle_Mixed_Fair.jpg'],
 ['/content/multimodel_effnet/data/Datasets2/Image_1277_Tile_Mixed_Damaged.jpg'],
 ['/content/multimodel_effnet/data/Datasets2/Image_6229_Metal_Gabled_Poor.jpg'],
 ['/content/multimodel_effnet/data/Datasets2/Image_27753_Tile_Mixed_Fair.jpg'],
 ['/content/multimodel_effnet/data/Datasets2/Image_25246_Metal_Gabled_Good.jpg']]

In [42]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [49]:
from model_deploy import pred_and_store,predict,pred_logits_to_labels,pred_label_2_classes,generate_out_response

In [106]:
def multimodel_effnet_gr(image, text,
                         model: torch.nn.Module=model,
                         transform:torchvision.transforms = test_transforms,
                         classifier: pipeline = classifier):
    # Replace this with your actual image and text processing logic
    #prompt1 = input("prompts")
    img = image.convert('RGB')
    img = transform(img).unsqueeze(0)
    # Put model into evaluation mode and turn on inference mode
    model.eval()
    with torch.inference_mode():
        # Pass the transformed image through the model and turn the prediction logits into prediction probabilities
        pred_logits = model(img)

    pred_label = pred_logits_to_labels(pred_logits)
    pred_classes = pred_label_2_classes(pred_label)
    #y_out = ['Fair','Shingle','Mixed','No','Low','No']
    result = generate_out_response(prompt=text, y_out=pred_classes,classifier = classifier)
    return result

In [ ]:
image = Image.open(example_list[3][0]).convert("RGB")
text = input("Promp: ")
image

In [107]:
print(multimodel_effnet_gr(image, text))

Image Analysis Results:

User Input: "Analyze all the attributes for the image"
Interpretation: 
  * The Roof Condition is Fair.
  * The Roof Material is Tile.
  * The Roof Style is Mixed.
  * The Solar Panel is No.
  * The Tree Overhang is No.
  * The Swimming Pool is Screened.

Suggestions: 
  * Your roof is in fair condition.
  * While the roof currently functional, consider these actions to prolong its lifespan.
  * schedule a professional inspection, perform proactive maintenance, and plan for future replacement.
  * Tile roofs are known for their longevity and beauty but can be more expensive.
  * Tile roofs are great chioce for warmer climates.
  * Mixed roof styles combine elements of different styles.
  * Mixed roof can be more complex, so it’s important to work with an experienced roofer.
  * If you are considering solar panels in the future, Give your installation requirements.
  * We offer SOTA solar design installations.
  * Using AI we predict which facets have more expos

In [52]:
import gradio as gr


iface = gr.Interface(
    fn=multimodel_effnet_gr,
    inputs=[gr.Image(type='pil'), "text"],
    outputs="text",
    examples=example_list,
    title="<b>TWIA: Building Attribute Classifier</b>",
    description="An EfficientNetB2 model for analyzing roof condition, materials, and more.",
    article="View the project on GitHub",
    css="""
        .gradio-container h1 { font-size: 22px; }
        .gradio-container a { color: #007bff; text-decoration: none; }
        .gradio-container a:hover { text-decoration: underline; }
    """,)

iface.launch(debug=True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://050fb9118482191cd2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://050fb9118482191cd2.gradio.live


In [ ]:
#article = "Created https://github.com/omer770/multimodel_effnet.git"

# Create the Gradio demo
demo = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil"),
    outputs=[
        gr.Label(label="Roof Condition"),
        gr.Label(label="Roof Material"),
        gr.Label(label="Roof Style"),
        gr.Label(label="Solar Panel"),
        gr.Label(label="Tree Overhang"),
        gr.Label(label="Swimming Pool"),
    ],
    examples=example_list,
    title="<b>TWIA: Building Attribute Classifier</b>",
    description="An EfficientNetB2 model for analyzing roof condition, materials, and more.",
    article="View the project on GitHub",
    css="""
        .gradio-container h1 { font-size: 22px; }
        .gradio-container a { color: #007bff; text-decoration: none; }
        .gradio-container a:hover { text-decoration: underline; }
    """,)

# Launch the demo!
demo.launch(debug=False, # print errors locally?
            share=True) # generate a publically shareable URL?

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://3510d499cf197d6e9a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7865 <> https://3510d499cf197d6e9a.gradio.live


In [54]:
label_2_item_lst

[{0: 'Fair', 1: 'Good', 2: 'Poor', 3: 'Damaged'},
 {0: 'Metal',
  1: 'Poly',
  2: 'Shingle',
  3: 'Tile',
  4: 'Ballasted',
  5: 'Asphalt'},
 {0: 'Flat', 1: 'Gabled', 2: 'Hip', 3: 'Mixed'},
 {0: 'No', 1: 'Yes'},
 {0: 'Low', 1: 'Medium', 2: 'High', 3: 'No'},
 {0: 'No', 1: 'Yes', 2: 'Screened'}]